In [15]:
import math
import numpy as np
import pandas as pd
from dateutil.parser import parse

pd.set_option('display.max_columns', None)

In [16]:
def loadCSVFile(csvPath, sepStyle, encodeStyle):
    return pd.read_csv(csvPath, sep=sepStyle, encoding=encodeStyle, low_memory=False)

In [17]:
def mergeDataFrames(leftDF, rightDF, keyColumnL, keyColumnR, suffixTuple):
    return leftDF.merge(rightDF, how='left', left_on=keyColumnL, right_on=keyColumnR, suffixes=suffixTuple)

In [18]:
def mergeDtHrUTI(listDtInUTI, listHrInUTI, listDtOutUTI, listHrOutUTI):
    listDTHR_InUTI, listDTHR_OutUTI= [], []
    
    for tplDtHrInUTI in zip(listDtInUTI, listHrInUTI):
        inDtUTI = tplDtHrInUTI[0]
        inHrUTI = tplDtHrInUTI[1]
        if type(inDtUTI) != str and type(inHrUTI) != str:
            if math.isnan(inDtUTI) == True or math.isnan(inHrUTI) == True:
                listDTHR_InUTI.append(np.nan)
        else:
            listDTHR_InUTI.append(' '.join(tplDtHrInUTI))
            
    for tplDtHrOutUTI in zip(listDtOutUTI, listHrOutUTI):
        outDtUTI = tplDtHrOutUTI[0]
        outHrUTI = tplDtHrOutUTI[1]
        if type(outDtUTI) != str and type(outHrUTI) != str:
            if math.isnan(outDtUTI) == True or math.isnan(outHrUTI) == True:
                listDTHR_OutUTI.append(np.nan)
        else:
            listDTHR_OutUTI.append(' '.join(tplDtHrOutUTI))
            
    return listDTHR_InUTI, listDTHR_OutUTI

In [19]:
def timeCounterBtwnDates(bDate, eDate):
    _bDate = parse(bDate, dayfirst=True).ctime()
    _bDate = parse(_bDate, dayfirst=True)
    
    _eDate = parse(eDate, dayfirst=True).ctime()
    _eDate = parse(_eDate, dayfirst=True)

    delta = _eDate - _bDate

    totalDays = delta.total_seconds()/86400
    
    return totalDays

In [20]:
def getDaysCountByRow(row, posREG, posALTA, posLEITO, posInUTI, posOutUTI):
    if type(row[posREG]) == str and type(row[posInUTI]) == str and type(row[posOutUTI]) == str:
        if type(row[posLEITO]) != str or row[posLEITO] == '':
           Pre_UTITime = timeCounterBtwnDates(bDate=row[posREG], eDate=row[posInUTI])
        else:
           Pre_UTITime = timeCounterBtwnDates(bDate=row[posLEITO], eDate=row[posInUTI])
        UTI_TIME = timeCounterBtwnDates(bDate=row[posInUTI], eDate=row[posOutUTI])
    else:
       if type(row[posREG]) == str and type(row[posALTA]) == str and type(row[posInUTI]) == float and type(row[posOutUTI]) == float:
          Pre_UTITime = timeCounterBtwnDates(bDate=row[posREG], eDate=row[posALTA])
          UTI_TIME = 0.0
       else:
          if type(row[posREG]) == float or type(row[posALTA]) == float:
             #print('Maybe PANIC here too!')
             Pre_UTITime = np.nan
             UTI_TIME = np.nan
    
#    if type(row[posREG]) == float and type(row[posALTA]) == str and type(row[posInUTI]) == float and type(row[posOutUTI]) == float:
#        Pre_UTITime = np.nan
#        UTI_TIME = np.nan
    
    return Pre_UTITime, UTI_TIME

In [21]:
def getTimeForEachDFRow(DFRows):
    list_PreUTITime, list_UTITime = [], []
    
    iREG=DF_REG_EST_UTI.columns.get_loc('DTHR_REGISTRO')
    iALTA=DF_REG_EST_UTI.columns.get_loc('DTHR_ALTA')
    iLEITO=DF_REG_EST_UTI.columns.get_loc('DTHR_LEITO')
    iInUTI=DF_REG_EST_UTI.columns.get_loc('DTHR_IN_UTI')
    iOutUTI=DF_REG_EST_UTI.columns.get_loc('DTHR_OUT_UTI')
    for row in DFRows:
      pUTI, UTI = getDaysCountByRow(row=row, posREG=iREG, posALTA=iALTA, posLEITO=iLEITO, posInUTI=iInUTI, posOutUTI=iOutUTI)
      list_PreUTITime.append(pUTI)
      list_UTITime.append(UTI)
    
    return list_PreUTITime, list_UTITime

In [22]:
if __name__ == '__main__':
    
    # Carrega os dados
    DF_REGISTRO = loadCSVFile(csvPath='./data/dataToPreProcessing/atendimentos.csv', encodeStyle='latin1', sepStyle=';')
    DF_ESTUDO = loadCSVFile(csvPath='./data/dataToPreProcessing/estudo.csv', encodeStyle='latin1', sepStyle=';')
    DF_UTI = loadCSVFile(csvPath='./data/dataToPreProcessing/uti.csv', encodeStyle='latin1', sepStyle=';')    
    
    #Remove duplicações
    DF_REGISTRO = DF_REGISTRO.drop_duplicates()
    DF_ESTUDO = DF_ESTUDO.drop_duplicates()
    DF_UTI = DF_UTI.drop_duplicates()

    #Corrige DF_UTI para que tenha uma entrada para cada passagem na UTI
#    print(DF_UTI.head(50))
    # Para corrigir DF_UTI, vamos precisar usar datas de entrada em leito (se vazio usamos DTHR_REGISTRO)
    DF_UTI['DTHR_LEITO'] = ''
    # Precisamos ordenar por atendimento e data-hora de entrada
    DF_UTI['DATA_HORA_ENTRADA'] = DF_UTI['DT_ENTRADA']+' '+DF_UTI['HR_ENTRADA']
    DF_UTI['DATA_HORA_ENTRADA'] = pd.to_datetime(DF_UTI.DATA_HORA_ENTRADA,format='%d/%m/%Y %H:%M:%S')
    DF_UTI = DF_UTI.sort_values(by=['CD_ATENDIMENTO','DATA_HORA_ENTRADA'],ignore_index=True)
#    print(DF_UTI.head(50))
    # Vamos precisar também contar a reentrada
    DF_UTI['REENTRADA']=0
    i = 0
    j = 0
    DF_UTI2 = DF_UTI.copy()
    while i < len(DF_UTI.index)-1 :
      if DF_UTI.loc[i,'CD_ATENDIMENTO'] == DF_UTI.loc[i+1,'CD_ATENDIMENTO'] :
        if type(DF_UTI.loc[i,'DT_SAIDA']) != str or type(DF_UTI.loc[i,'HR_SAIDA']) != str :
          #HORROR: we don't know why this is happening
          print('PANIC!!!!!!!!!!!!')
          i=i+1
          j=i
        if timeCounterBtwnDates(bDate=DF_UTI.loc[i,'DT_SAIDA']+' '+DF_UTI.loc[i,'HR_SAIDA'], eDate=DF_UTI.loc[i+1,'DT_ENTRADA']+' '+DF_UTI.loc[i+1,'HR_ENTRADA']) < 1 :
#          print('merging two rows at: ',i)
          DF_UTI2.loc[j,'DT_SAIDA'] = DF_UTI.loc[i+1,'DT_SAIDA']
          DF_UTI2.loc[j,'HR_SAIDA'] = DF_UTI.loc[i+1,'HR_SAIDA']
          DF_UTI2 = DF_UTI2.drop([i+1])
          i = i+1
        else :
#          print('spliting UTI entrance at: ',i)
          DF_UTI2.loc[i+1,'DTHR_LEITO']=DF_UTI.loc[i,'DT_SAIDA']+' '+DF_UTI.loc[i,'HR_SAIDA']
          DF_UTI2.loc[i+1,'REENTRADA']=DF_UTI2.loc[j,'REENTRADA']+1
          i = i+1
          j = i
      else :
        i = i+1
        j = i

    DF_UTI = DF_UTI2

    #print(DF_UTI.head(50))

    #União das Tabelas REGISTRO e ESTUDO
    DF_REG_EST = mergeDataFrames(leftDF=DF_REGISTRO,
                                 rightDF=DF_ESTUDO,
                                 keyColumnL='CD_PACIENTE', keyColumnR='CD_PACIENTE_FROM_ESTUDO',
                                 suffixTuple=('_FROM_REG', '_FROM_EST'))
    
    # It seems the datatypes of the CD_ATENDIMENTO in different tables are not the same
    DF_REG_EST['CD_ATENDIMENTO'] = pd.to_numeric(DF_REG_EST['CD_ATENDIMENTO'])
    DF_UTI['CD_ATENDIMENTO'] = pd.to_numeric(DF_UTI['CD_ATENDIMENTO'])
    # Also sort before merging with UTI so we can check if it's working more easily
    DF_REG_EST = DF_REG_EST.sort_values(by=['CD_ATENDIMENTO'],ignore_index=True)

    #União das Tabelas REGISTRO-ESTUDO e UTI
    DF_REG_EST_UTI = mergeDataFrames(leftDF=DF_REG_EST,
                                     rightDF=DF_UTI,
                                     keyColumnL='CD_ATENDIMENTO', keyColumnR='CD_ATENDIMENTO',
                                     suffixTuple=('_FROM_MRG_REG_EST', '_FROM_UTI'))
    
    
    #Remove duplicações
    DF_REG_EST_UTI = DF_REG_EST_UTI.drop_duplicates()
    
    
    #União dos campos DT_ENTRADA + HR_ENTRADA e DT_SAIDA + HR_SAIDA
    lDtInUTI = DF_REG_EST_UTI['DT_ENTRADA'].values.tolist()
    lHrInUTI = DF_REG_EST_UTI['HR_ENTRADA'].values.tolist()
    lDtOutUTI = DF_REG_EST_UTI['DT_SAIDA'].values.tolist()
    lHrOutUTI = DF_REG_EST_UTI['HR_SAIDA'].values.tolist()
    
    listDTHR_InUTI, listDTHR_OutUTI = mergeDtHrUTI(listDtInUTI=lDtInUTI,
                                                   listHrInUTI=lHrInUTI,
                                                   listDtOutUTI=lDtOutUTI,
                                                   listHrOutUTI=lHrOutUTI)
    
    #Adicionando União anterior a tabela geral
    DF_REG_EST_UTI['DTHR_IN_UTI'] = listDTHR_InUTI
    DF_REG_EST_UTI['DTHR_OUT_UTI'] = listDTHR_OutUTI
    
    #Calculando e Adicionando numero de dias antes e durante a UTI
    list_PreUTITime, list_UTITime = getTimeForEachDFRow(DFRows=DF_REG_EST_UTI.values.tolist())
    
    DF_REG_EST_UTI['DIAS_PRE_UTI'] = list_PreUTITime
    DF_REG_EST_UTI['DIAS_NA_UTI'] = list_UTITime

#    i=0
#    j=0
#    while i < len(DF_UTI.index) :
#      if DF_REG_EST_UTI.loc[i,'DIAS_PRE_UTI'] < 0 or DF_REG_EST_UTI.loc[i,'DIAS_PRE_UTI'] < 0 :
#        j = j+1
#      i=i+1
#    print('ERROS ',j)
    # 228 linhas com numero negativos
    # ATTENTION DELETING NEGATIVE CALCULATED NUMBERS
    DF_REG_EST_UTI = DF_REG_EST_UTI[DF_REG_EST_UTI.DIAS_PRE_UTI>=0]
    DF_REG_EST_UTI = DF_REG_EST_UTI[DF_REG_EST_UTI.DIAS_NA_UTI>=0]
    print(DF_REG_EST_UTI.head(10))

    #print(DF_REG_EST['CD_ATENDIMENTO'].head(10))
    #print(DF_UTI.head(10))
    #print(DF_REG_EST_UTI['CD_ATENDIMENTO'].head(10),DF_REG_EST_UTI['REENTRADA'].head(10))


      ID        DTHR_REGISTRO    PORTA_ENTRADA DATA_INICIO_SINTOMAS  \
2   4463  08/07/2020 12:56:20              NaN                  NaN   
3   4463  08/07/2020 12:56:20              NaN                  NaN   
4   4463  08/07/2020 12:56:20              NaN                  NaN   
5   4463  08/07/2020 12:56:20              NaN                  NaN   
15  1149  27/03/2020 14:03:00              NaN                  NaN   
16  1014  04/03/2020 13:59:22      PPUC adulto  29/02/2020 00:00:00   
18   714  13/04/2020 11:35:08      PPUC adulto  07/04/2020 00:00:00   
19  8743  20/08/2020 20:31:00              NaN                  NaN   
23   998  06/03/2020 13:12:48  PPUC pediátrico                  NaN   
26  1066  03/04/2020 16:02:12      PPUC adulto  28/03/2020 00:00:00   

   HISTORICO_VIAGEM                      PAIS_VISITADO  \
2               NaN                                NaN   
3               NaN                                NaN   
4               NaN                         

In [23]:
#getDaysCountByRow(DF_REG_EST_UTI.values.tolist()[7], posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)

In [24]:
#print('REG', DF_REG_EST_UTI.values.tolist()[7][1])

In [25]:
DF_REG_EST_UTI.to_csv(r'./data/PosProcessing-v0.2.2/Final_Dataset.csv', index=False)

In [14]:
DF_REG_EST_UTI

,ID,DTHR_REGISTRO,PORTA_ENTRADA,DATA_INICIO_SINTOMAS,HISTORICO_VIAGEM,PAIS_VISITADO,CIDADES_VISITADAS,DATA_RETORNO,COLETOU_AMOSTRA,LOCAL_COLETA,DATA_COLETA,PCR_INFLUENZA,IFI_OUTRO_VIRUS,PCR_COVID_19,STATUS_COVID_FROM_MRG_REG_EST,STATUS_COVID_ORDEM,DESFECHO,CD_ATENDIMENTO,DS_TP_ATENDIMENTO,CD_PACIENTE,GENERO,IDADE,SN_OBITO,DIAGNOSTICO_SARS,OBITO_SARS,TIPO_SAIDA,TIPO_OBITO,DTHR_ALTA,CD_CID,DS_ESPECIALID_SAI,REINTER_30D,CARATER_ATENDIMENTO,EMERG_SERVICO,EMERG_RISCO,CID_DIAG_ATEND,CID_CAPITULO_DIAG,BLOCO_CENTRAL_ST,HEMODINAMICA_ST,EX_LAB_ST,EX_IMG_ST,EX_OUTROS_ST,CD_PACIENTE_FROM_ESTUDO,COMPETENCIA,PORTA_ENTRADA_FROM_ESTUDO,DATA_INICIO_SINTOMAS_FROM_ESTUDO,HISTORICO_VIAGEM_FROM_ESTUDO,CIDADES_VISITADAS_FROM_ESTUDO,LOCAL_COLETA_FROM_ESTUDO,DATA_COLETA_FROM_ESTUDO,PCR_COVID_19_FROM_ESTUDO,DESFECHO_FROM_ESTUDO,TIPO_REGISTRO,SN_IFI_OV_SINCICIAL_RESP,SN_IFI_OV_PARAINFLU3,SN_IFI_OV_PARAINFLU2,SN_IFI_OV_PARAINFLU1,SN_IFI_OV_INFLUA,SN_IFI_OV_INFLUB,SN_IFI_OV_ADENOVIRUS,SN_CRIT_CONTATO_CASO_COVID,SN_CRIT_CONTATO_CASO_SUSPEITO,SN_CRIT_PROF_SAUDE_HOSPITAL,SN_CRIT_PROF_SAUDE_OUT,SN_CRIT_SRAG,SN_CRIT_SINDROME_GRIPAL,SN_CRIT_IGNORADO,SN_CRIT_OUTROS,SN_CRIT_PROT_COVID19,ESTUDO_TP_EXPOSICAO,ESTUDO_COMORB_PREV_HAS,ESTUDO_COMORB_PREV_IAM,ESTUDO_COMORB_PREV_IC,ESTUDO_COMORB_PREV_DRC,ESTUDO_COMORB_PREV_DMI,ESTUDO_COMORB_PREV_DMII,ESTUDO_COMORB_PREV_CARCINOMA,ESTUDO_COMORB_PREV_ADENOMA,ESTUDO_COMORB_PREV_LINFOMA,ESTUDO_COMORB_PREV_AVC,ESTUDO_COMORB_PREV_ASMA,ESTUDO_COMORB_PREV_DPOC,ESTUDO_COMORB_PREV_OBESIDADE,ESTUDO_COMORB_PREV_EPILEPSIA,ESTUDO_COMORB_PREV_HIGIDO,ESTUDO_COMORB_PREV_OUTRAS,ESTUDO_CHARLSON,ESTUDO_FR_DIA_COLETA,ESTUDO_FC_DIA_COLETA,ESTUDO_PA_DIA_COLETA,ESTUDO_TEMPERATURA_DIA_COLETA,ESTUDO_SATURACAO_DIA_COLETA,ESTUDO_APACHE2,ESTUDO_RISCO,ESTUDO_TRATAMENTO,ESTUDO_COMORB_PREV_TX_ORG_SOL,ESTUDO_CONDUTA,ESTUDO_LOCAL_CONTAMINACAO,ESTUDO_LOCAL_CONTAM_UI,ESTUDO_DT_DESFECHO,ESTUDO_COMORB_PREV_DESCON,ESTUDO_COMORB_PREV_DVP,ESTUDO_COMORB_PREV_DEMENCIA,ESTUDO_COMORB_PREV_DTC,ESTUDO_COMORB_PREV_UP,ESTUDO_COMORB_PREV_DFL,ESTUDO_COMORB_PREV_DFM,ESTUDO_COMORB_PREV_HEMIPLEGIA,ESTUDO_COMORB_PREV_LEUCEMIA,ESTUDO_COMORB_PREV_HIV,ESTUDO_COMORB_PREV_TUL,ESTUDO_COMORB_PREV_TUM,STATUS_COVID_FROM_UTI,CD_PACIENTE_FROM_UTI,CD_LEITO,CD_UNIDADE,DS_UNIDADE,DT_ENTRADA,HR_ENTRADA,DT_SAIDA,HR_SAIDA,DTHR_LEITO,DATA_HORA_ENTRADA,REENTRADA,DTHR_IN_UTI,DTHR_OUT_UTI,DIAS_PRE_UTI,DIAS_NA_UTI
2,4463,08/07/2020 12:56:20,NaN,NaN,NaN,NaN,NaN,NaN,SIM,9 N - INTERNACAO,08/07/2020,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,925313.0,INTERNADO,454612.0,MASCULINO,"42,498401826484",N,NÃO,NÃO,ALTA,NaN,07/10/2020 13:13:46,K263,NEUROCIRURGIA,NÃO,URGÊNCIA,PRONTOPUC,LARANJA,K263,Aparelho digestivo,SIM,SIM,SIM,SIM,SIM,454612.0,01/07/2020,NaN,NaN,NaN,NaN,9 N - INTERNACAO,08/07/2020,NEGATIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,HOSPITALAR HSL,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NEGATIVOS,454612.0,1607.0,7.0,UTI,07/12/2019,11:51:59,18/12/2019,13:48:48,26/11/2019 17:53:24,2019-12-07 11:51:59,1.0,07/12/2019 11:51:59,18/12/2019 13:48:48,10.749016,11.081123
3,4463,08/07/2020 12:56:20,NaN,NaN,NaN,NaN,NaN,NaN,SIM,9 N - INTERNACAO,08/07/2020,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,925313.0,INTERNADO,454612.0,MASCULINO,"42,498401826484",N,NÃO,NÃO,ALTA,NaN,07/10/2020 13:13:46,K263,NEUROCIRURGIA,NÃO,URGÊNCIA,PRONTOPUC,LARANJA,K263,Aparelho digestivo,SIM,SIM,SIM,SIM,SIM,454612.0,01/07/2020,NaN,NaN,NaN,NaN,9 N - INTERNACAO,08/07/2020,NEGATIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,HOSPITALAR HSL,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NEGATIVOS,454612.0,1613.0,7.0,UTI,10/01/2020,05:43:14,03/02/2020,14:04:26,18/12/2019 13:48:48,2020-01-10 05:43:14,2.0,10/01/2020 05:43:14,03/02/2020 14:04:26,22.662801,24.348056
4,4463,08/07/2020 12:56:20,NaN,NaN,NaN,NaN,NaN,NaN,SIM,9 N - INTERNACAO,08/07/2020,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,925313.0,INT

In [ ]:
#print('ALTA', DF_REG_EST_UTI.values.tolist()[7][27])

ALTA 13/03/2021 13:18:53


In [ ]:
#print('InUTI', DF_REG_EST_UTI.values.tolist()[7][119])

InUTI nan


In [ ]:
#print('OutUTI', DF_REG_EST_UTI.values.tolist()[7][120])

OutUTI nan


In [ ]:
DF_REG_EST_UTI.loc[DF_REG_EST_UTI['CD_ATENDIMENTO']==1584472.0]

,ID,DTHR_REGISTRO,PORTA_ENTRADA,DATA_INICIO_SINTOMAS,HISTORICO_VIAGEM,PAIS_VISITADO,CIDADES_VISITADAS,DATA_RETORNO,COLETOU_AMOSTRA,LOCAL_COLETA,DATA_COLETA,PCR_INFLUENZA,IFI_OUTRO_VIRUS,PCR_COVID_19,STATUS_COVID_FROM_MRG_REG_EST,STATUS_COVID_ORDEM,DESFECHO,CD_ATENDIMENTO,DS_TP_ATENDIMENTO,CD_PACIENTE,GENERO,IDADE,SN_OBITO,DIAGNOSTICO_SARS,OBITO_SARS,TIPO_SAIDA,TIPO_OBITO,DTHR_ALTA,CD_CID,DS_ESPECIALID_SAI,REINTER_30D,CARATER_ATENDIMENTO,EMERG_SERVICO,EMERG_RISCO,CID_DIAG_ATEND,CID_CAPITULO_DIAG,BLOCO_CENTRAL_ST,HEMODINAMICA_ST,EX_LAB_ST,EX_IMG_ST,EX_OUTROS_ST,CD_PACIENTE_FROM_ESTUDO,COMPETENCIA,PORTA_ENTRADA_FROM_ESTUDO,DATA_INICIO_SINTOMAS_FROM_ESTUDO,HISTORICO_VIAGEM_FROM_ESTUDO,CIDADES_VISITADAS_FROM_ESTUDO,LOCAL_COLETA_FROM_ESTUDO,DATA_COLETA_FROM_ESTUDO,PCR_COVID_19_FROM_ESTUDO,DESFECHO_FROM_ESTUDO,TIPO_REGISTRO,SN_IFI_OV_SINCICIAL_RESP,SN_IFI_OV_PARAINFLU3,SN_IFI_OV_PARAINFLU2,SN_IFI_OV_PARAINFLU1,SN_IFI_OV_INFLUA,SN_IFI_OV_INFLUB,SN_IFI_OV_ADENOVIRUS,SN_CRIT_CONTATO_CASO_COVID,SN_CRIT_CONTATO_CASO_SUSPEITO,SN_CRIT_PROF_SAUDE_HOSPITAL,SN_CRIT_PROF_SAUDE_OUT,SN_CRIT_SRAG,SN_CRIT_SINDROME_GRIPAL,SN_CRIT_IGNORADO,SN_CRIT_OUTROS,SN_CRIT_PROT_COVID19,ESTUDO_TP_EXPOSICAO,ESTUDO_COMORB_PREV_HAS,ESTUDO_COMORB_PREV_IAM,ESTUDO_COMORB_PREV_IC,ESTUDO_COMORB_PREV_DRC,ESTUDO_COMORB_PREV_DMI,ESTUDO_COMORB_PREV_DMII,ESTUDO_COMORB_PREV_CARCINOMA,ESTUDO_COMORB_PREV_ADENOMA,ESTUDO_COMORB_PREV_LINFOMA,ESTUDO_COMORB_PREV_AVC,ESTUDO_COMORB_PREV_ASMA,ESTUDO_COMORB_PREV_DPOC,ESTUDO_COMORB_PREV_OBESIDADE,ESTUDO_COMORB_PREV_EPILEPSIA,ESTUDO_COMORB_PREV_HIGIDO,ESTUDO_COMORB_PREV_OUTRAS,ESTUDO_CHARLSON,ESTUDO_FR_DIA_COLETA,ESTUDO_FC_DIA_COLETA,ESTUDO_PA_DIA_COLETA,ESTUDO_TEMPERATURA_DIA_COLETA,ESTUDO_SATURACAO_DIA_COLETA,ESTUDO_APACHE2,ESTUDO_RISCO,ESTUDO_TRATAMENTO,ESTUDO_COMORB_PREV_TX_ORG_SOL,ESTUDO_CONDUTA,ESTUDO_LOCAL_CONTAMINACAO,ESTUDO_LOCAL_CONTAM_UI,ESTUDO_DT_DESFECHO,ESTUDO_COMORB_PREV_DESCON,ESTUDO_COMORB_PREV_DVP,ESTUDO_COMORB_PREV_DEMENCIA,ESTUDO_COMORB_PREV_DTC,ESTUDO_COMORB_PREV_UP,ESTUDO_COMORB_PREV_DFL,ESTUDO_COMORB_PREV_DFM,ESTUDO_COMORB_PREV_HEMIPLEGIA,ESTUDO_COMORB_PREV_LEUCEMIA,ESTUDO_COMORB_PREV_HIV,ESTUDO_COMORB_PREV_TUL,ESTUDO_COMORB_PREV_TUM,STATUS_COVID_FROM_UTI,CD_PACIENTE_FROM_UTI,CD_LEITO,CD_UNIDADE,DS_UNIDADE,DT_ENTRADA,HR_ENTRADA,DT_SAIDA,HR_SAIDA,DTHR_IN_UTI,DTHR_OUT_UTI,DIAS_PRE_UTI,DIAS_NA_UTI
2,25975,08/03/2021 08:11:00,NaN,NaN,NaN,NaN,NaN,NaN,SIM,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,1584472.0,INTERNADO,1832839.0,MASCULINO,"58,8792237442922",S,NÃO,NÃO,ÓBITO,NaN,21/03/2021 18:59:44,A310,CIRURGIA GERAL,NÃO,URGÊNCIA,PRONTOSUS,LARANJA,A419,Infecciosas e parasitárias,SIM,NÃO,SIM,SIM,SIM,1832839.0,08/03/2021,NaN,NaN,NaN,NaN,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NEGATIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NEGATIVOS,1832839.0,1649.0,8.0,UTI,08/03/2021,13:31:53,18/03/2021,23:22:48,08/03/2021 13:31:53,18/03/2021 23:22:48,112.633194,-112.410359


In [ ]:
#getDaysCountByRow(DF_REG_EST_UTI.values.tolist()[2], posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)

(112.63319444444444, -112.4103587962963)

In [ ]:
#print('REG', DF_REG_EST_UTI.values.tolist()[2][1])

REG 08/03/2021 08:11:00


In [ ]:
#print('ALTA', DF_REG_EST_UTI.values.tolist()[2][27])

ALTA 21/03/2021 18:59:44


In [ ]:
#print('InUTI', DF_REG_EST_UTI.values.tolist()[2][119])

InUTI 23:22:48


In [ ]:
#print('OutUTI', DF_REG_EST_UTI.values.tolist()[2][120])

OutUTI 08/03/2021 13:31:53
